In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np



In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split
import cv2

# Set paths and parameters
data_dir = '../Data/images_original'  # Root folder containing genre subfolders
image_size = (128, 128)  # Resize images to this size
batch_size = 32
num_classes = 0  # Will be determined dynamically
genres = []

# Step 1: Load and preprocess images
def load_images_and_labels():
    images = []
    labels = []
    label_map = {}  # Map genre to integer label
    
    # Get list of genre subfolders
    for genre in os.listdir(data_dir):
        genre_path = os.path.join(data_dir, genre)
        if os.path.isdir(genre_path):
            genres.append(genre)
            label_map[genre] = len(label_map)  # Assign integer to each genre
            
            # Load images from genre subfolder
            for filename in os.listdir(genre_path):
                if filename.endswith('.png'):
                    img_path = os.path.join(genre_path, filename)
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.resize(img, image_size)  # Resize image
                        img = img / 255.0  # Normalize pixel values to [0, 1]
                        images.append(img)
                        labels.append(label_map[genre])
    
    return np.array(images), np.array(labels), label_map

# Load data
images, labels, label_map = load_images_and_labels()
num_classes = len(label_map)

# Step 2: Shuffle images and labels together
# Create an array of indices and shuffle them
indices = np.arange(len(images))
np.random.seed(42)  # For reproducibility
np.random.shuffle(indices)

# Apply shuffled indices to images and labels
images = images[indices]
labels = labels[indices]


# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Step 2: Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Step 3: Data augmentation (optional, to improve generalization)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Fit the model
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          epochs=20,
          validation_data=(X_test, y_test))

# Step 4: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Optional: Make predictions on a few test images
predictions = model.predict(X_test[:5])
predicted_labels = [list(label_map.keys())[np.argmax(pred)] for pred in predictions]
actual_labels = [list(label_map.keys())[label] for label in y_test[:5]]
print("Sample Predictions:")
for i in range(5):
    print(f"Image {i+1}: Predicted={predicted_labels[i]}, Actual={actual_labels[i]}")

Epoch 1/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 270ms/step - accuracy: 0.1028 - loss: 2.3943 - val_accuracy: 0.1200 - val_loss: 2.2893
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 266ms/step - accuracy: 0.1224 - loss: 2.2810 - val_accuracy: 0.1650 - val_loss: 2.2502
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 268ms/step - accuracy: 0.1574 - loss: 2.2453 - val_accuracy: 0.2000 - val_loss: 2.1519
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - accuracy: 0.1988 - loss: 2.1841 - val_accuracy: 0.2000 - val_loss: 2.0518
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 249ms/step - accuracy: 0.1942 - loss: 2.1357 - val_accuracy: 0.2150 - val_loss: 2.1179
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 251ms/step - accuracy: 0.1902 - loss: 2.0971 - val_accuracy: 0.2500 - val_loss: 2.1106
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 252ms/step - accuracy: 0.2252 - loss: 2.0021 - val_accuracy: 0.3200 - val_loss: 1.8790
Epoch 8/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 254ms/step - accuracy: 0.2572 - loss: 2.0306 - val_accuracy: 0.

In [11]:
from sklearn.metrics import confusion_matrix, classification_report
y_pred = model.predict(X_test).argmax(axis=1)
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=list(label_map.keys())))

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Confusion Matrix:
[[ 0  3  1  0  0 13  0  3  4  1]
 [ 0 19  0  0  0  4  0  0  0  0]
 [ 0  0  3  0  0 22  0  2  0  0]
 [ 0  1  2  0  0  1  0  7  4  1]
 [ 0  0  1  0  1  1  1  0 15  0]
 [ 0  1  0  0  0 18  0  0  0  0]
 [ 0  0  0  1  0  0 13  1  3  3]
 [ 0  1  0  1  1  3  0  3  4  0]
 [ 0  0  0  0  0  2  0  3 16  1]
 [ 0  0  4  1  0  0  0  1  1  8]]

Classification Report:
              precision    recall  f1-score   support

       blues       0.00      0.00      0.00        25
   classical       0.76      0.83      0.79        23
     country       0.27      0.11      0.16        27
       disco       0.00      0.00      0.00        16
      hiphop       0.50      0.05      0.10        19
        jazz       0.28      0.95      0.43        19
       metal       0.93      0.62      0.74        21
         pop       0.15      0.23      0.18        13
      reggae       0.34      0.73      0.46        22
        rock       0.57      0.53      0.55     

c:\Users\botla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\botla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\botla\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [12]:
# Optional: Manual accuracy calculation
manual_accuracy = np.mean(y_pred == y_test)
print(f"Manually Calculated Accuracy: {manual_accuracy * 100:.2f}%")

Manually Calculated Accuracy: 40.50%
